In [2]:
import os as os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import subprocess

In [4]:
file_path_test = "/Users/gimle/Data 201/Data 201 project 3/fashion-mnist_test.csv"
file_path_train = "/Users/gimle/Data 201/Data 201 project 3/fashion-mnist_train.csv"

In [6]:
# pytorch and nn libraries: 

try:
    import torch
    print("torch is installed.")
except ImportError:
    print("Torch is not installed. Installing now...")
    subprocess.check_call(["pip", "install", "torch"])

torch is installed.


In [8]:
try:
    import kaggle
    print("Kaggle is installed.")
except ImportError:
    print("Kaggle is not installed. Installing now...")
    subprocess.check_call(["pip", "install", "kaggle"])

Kaggle is installed.


In [10]:

from kaggle.api.kaggle_api_extended import KaggleApi

if os.path.exists(file_path_train):
    print("train file correctly installed", file_path_train)
    
# if kaggle.json exists 
elif os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")):
    print("File does not exist, but now downloading from kaggle")
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files("zalando-research/fashionmnist", path='.', unzip=True)
    if os.path.exists(file_path_train):
        print("File is here:", file_path_train)
    else:
        print("File in the wrong place pls check.")
else: 
    print("please download the file / get kaggle API: https://www.kaggle.com/docs/api") 

train file correctly installed /Users/gimle/Data 201/Data 201 project 3/fashion-mnist_train.csv


In [12]:

if os.path.exists(file_path_train): 
    zalando_train = pd.read_csv(file_path_train)
    print("zim zalandobim!")
   
else: 
    print("youve got fix the filepath") 

zim zalandobim!


In [14]:
if os.path.exists(file_path_test): 
    zalando_test = pd.read_csv(file_path_test)
    print("zim zalandobim!")
   
else: 
    print("youve got fix the filepath") 

zim zalandobim!


In [16]:
zalando_train.head(3)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0


In [18]:
zalando_test.head(10)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,44,105,44,10,...,105,64,30,0,0,0,0,0,0,0
6,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,6,0,0,0,0,0,0,0,1,0,...,174,136,155,31,0,1,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,57,70,28,0,2,0,0,0,0,0


In [20]:
zalando_test.columns

Index(['label', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779', 'pixel780',
       'pixel781', 'pixel782', 'pixel783', 'pixel784'],
      dtype='object', length=785)

In [22]:
zalando_train.columns

Index(['label', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779', 'pixel780',
       'pixel781', 'pixel782', 'pixel783', 'pixel784'],
      dtype='object', length=785)

In [24]:
test_count = zalando_test['label'].value_counts()
print(test_count) 

label
0    1000
1    1000
2    1000
3    1000
8    1000
6    1000
5    1000
4    1000
7    1000
9    1000
Name: count, dtype: int64


In [26]:
train_count = zalando_train['label'].value_counts()
print(train_count) 

label
2    6000
9    6000
6    6000
0    6000
3    6000
4    6000
5    6000
8    6000
7    6000
1    6000
Name: count, dtype: int64


In [28]:
print("The trianing dataset accounts for approximately" , f"{(1/7)*100:.2f}" , "percent of the train + test data")

The trianing dataset accounts for approximately 14.29 percent of the train + test data


In [34]:
pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 3.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [32]:
# Importing necessary torch packages
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torchmetrics
from torchmetrics import Accuracy

ModuleNotFoundError: No module named 'torch.metrics'

## Formatting data: 

In [230]:
# Separate features and labels

features = zalando_train.drop(columns=["label"]).values / 255.0
target = zalando_train["label"].values

features_test = zalando_test.drop(columns=["label"]).values / 255.0
target_test = zalando_test["label"].values

# Convert to PyTorch tensors

features_tensor_train = torch.tensor(features, dtype=torch.float32)
target_tensor_train = torch.tensor(target, dtype=torch.long)

features_tensor_test = torch.tensor(features_test, dtype=torch.float32)
target_tensor_test = torch.tensor(target_test, dtype=torch.long)

In [232]:
# Tensor formatting complete after this step
train_dataset = TensorDataset(features_tensor_train, target_tensor_train)
test_dataset = TensorDataset(features_tensor_test, target_tensor_test)

# Check batch size
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

## Creating a neural network for multiclass classification

In [248]:
# The neural network: 
model = nn.Sequential(
    nn.Linear(784, 512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, 256), # extra layer post success
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 128), # extra layer post success
    nn.ReLU(), 
    nn.Linear(128, 10)
    )



## Loss function and optimizer

In [251]:
# Define Criterion and optimizer
criterion = nn.CrossEntropyLoss()  

# alternative: optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# includes learning rate
optimizer = optim.SGD(model.parameters(), lr=0.037, momentum=0.917)
optimizer.step()

## Training the model

In [254]:
# Training loop
num_epochs = 100 

for epoch in range(num_epochs): 
    model.train()
    running_loss = 0.0
    
    for features, labels in train_loader: 
        # Clearing gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(features)
        # Compute loss and gradients
        loss = criterion(outputs, labels)
        # Backwards pass
        loss.backward()
        # Update params
        optimizer.step()
        # Print update

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

Epoch 1/100, Loss: 0.8147
Epoch 2/100, Loss: 0.4516
Epoch 3/100, Loss: 0.3970
Epoch 4/100, Loss: 0.3697
Epoch 5/100, Loss: 0.3416
Epoch 6/100, Loss: 0.3231
Epoch 7/100, Loss: 0.3087
Epoch 8/100, Loss: 0.3001
Epoch 9/100, Loss: 0.2880
Epoch 10/100, Loss: 0.2792
Epoch 11/100, Loss: 0.2719
Epoch 12/100, Loss: 0.2615
Epoch 13/100, Loss: 0.2549
Epoch 14/100, Loss: 0.2484
Epoch 15/100, Loss: 0.2402
Epoch 16/100, Loss: 0.2362
Epoch 17/100, Loss: 0.2259
Epoch 18/100, Loss: 0.2260
Epoch 19/100, Loss: 0.2211
Epoch 20/100, Loss: 0.2181
Epoch 21/100, Loss: 0.2115
Epoch 22/100, Loss: 0.2051
Epoch 23/100, Loss: 0.2035
Epoch 24/100, Loss: 0.2004
Epoch 25/100, Loss: 0.1936
Epoch 26/100, Loss: 0.1907
Epoch 27/100, Loss: 0.1864
Epoch 28/100, Loss: 0.1816
Epoch 29/100, Loss: 0.1783
Epoch 30/100, Loss: 0.1740
Epoch 31/100, Loss: 0.1733
Epoch 32/100, Loss: 0.1717
Epoch 33/100, Loss: 0.1678
Epoch 34/100, Loss: 0.1607
Epoch 35/100, Loss: 0.1625
Epoch 36/100, Loss: 0.1575
Epoch 37/100, Loss: 0.1589
Epoch 38/1

### Calculating success / metrics

In [259]:
# Set the model to evaluation mode
model.eval()
validation_loss = 0.0
accuracy_metric = Accuracy(task="multiclass", num_classes=10)

with torch.no_grad():
  
  for features, labels in test_loader:
      outputs = model(features)
      loss = criterion(outputs, labels)
      validation_loss += loss.item()
      # Accuracy
      accuracy_metric.update(outputs, labels)

validation_loss_epoch = validation_loss/len(test_loader)
accuracy = accuracy_metric.compute()

print(f"Validation Loss: {validation_loss_epoch:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

# Set the model back to training mode
model.train()



accuracy_metric.reset()

Validation Loss: 0.5627
Validation Accuracy: 0.8981


In [ ]:
# Implement scheduler?  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
# Add nn.Dropout(0.5) to reduce overfitting


In [119]:
# Create accuracy metric using torch metrics
metric = torchmetrics.Accuracy(task="multiclass", num_classes=10)
for features, labels in train_loader:
    outputs = model(features)
    
    # Calculate accuracy over the batch
    acc = metric(outputs.softmax(dim=-1), labels)
    
# Calculate accuracy over the whole epoch
acc = metric.compute()

# Reset the metric for the next epoch 
metric.reset()
plot_errors(model, train_loader)

NameError: name 'plot_errors' is not defined